# 4.2 模型参数的访问、初始化和共享

In [101]:
import torch
from torch import nn
from torch.nn import init

print(torch.__version__)

1.3.1


In [102]:
net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1))  # pytorch已进行默认初始化

print(net)
X = torch.rand(2, 4)
Y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


## 4.2.1 访问模型参数

In [103]:
print(type(net.named_parameters()))
for name, param in net.named_parameters():
    print(name, param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [104]:
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [105]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        self.weight2 = torch.rand(20, 20)
    def forward(self, x):
        pass
    
n = MyModel()
for name, param in n.named_parameters():
    print(name)

weight1


In [106]:
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad)
Y.backward()
print(weight_0.grad)

tensor([[-0.1900,  0.2341, -0.2687,  0.2384],
        [-0.3007, -0.3874, -0.1653, -0.0692],
        [ 0.4688,  0.3697,  0.1423, -0.3145]])
None
tensor([[-0.2986, -0.1218, -0.0942, -0.2159],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0995,  0.0406,  0.0314,  0.0720]])


## 4.2.2 初始化模型参数

In [107]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)

0.weight tensor([[-9.8088e-05,  3.3216e-04,  5.8272e-03,  2.5732e-03],
        [ 1.9502e-02,  4.6102e-04,  2.7263e-03,  1.7079e-02],
        [-3.4961e-03, -1.3951e-02, -5.8859e-03, -1.6801e-02]])
2.weight tensor([[ 0.0088,  0.0207, -0.0161]])


In [108]:
for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


## 4.2.3 自定义初始化方法

In [109]:
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10, 10)
        tensor *= (tensor.abs() >= 5).float()

In [110]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)

0.weight tensor([[ 0.0000, -0.0000,  7.8850,  7.6932],
        [ 0.0000,  0.0000, -8.4536,  0.0000],
        [-7.1903, -0.0000, -0.0000, -0.0000]])
2.weight tensor([[-0.0000, 5.8327, 7.8745]])


In [111]:
for name, param in net.named_parameters():
    if 'bias' in name:
        param.data += 1
        print(name, param.data)

0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


## 4.2.4 共享模型参数

In [116]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear) 
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)

Linear(in_features=1, out_features=1, bias=False)
Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [113]:
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

True
True


In [115]:
x = torch.ones(1, 1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad)

tensor([[1.]])
tensor(9., grad_fn=<SumBackward0>)
tensor([[12.]])
